In [1]:
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.display import display
from IPython.utils.capture import capture_output
from pprint import pprint
from pathlib import Path

In [2]:
@magics_class
class CaptureMagic(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "--path",
        "-p",
        default=None,
        help=(
            "The path where the video will be saved to. When there is more then one video, multiple paths have to be defined"
        ),
    )
    @cell_magic
    def capture_video(self, line, cell):
        args = magic_arguments.parse_argstring(CaptureMagic.capture_video, line)
        paths = args.path.strip('"').split(" ")
        with capture_output(stdout=False, stderr=False, display=True) as result:
            self.shell.run_cell(cell)
        for output in result.outputs:
            display(output) # only disabled for debugging
            global data # for debugging 
            data = output.data

            pprint(data) # for debugging 

            print("#####")
            if "text/html" in data: # this is not nice, is there any better way to access IPython.core.display.Video object ?
                path = paths.pop(0)
                if not path:
                    raise ValueError("Too few paths given!")
                video_object = data["text/html"]
                split_string = video_object.split('"')
                video_url = split_string[1]
                print(video_url) # for debugging 
                print(path) # for debugging 

                dest = Path(path)
                src = Path(video_url)
                dest.write_bytes(src.read_bytes())
                

                
ipy = get_ipython()
ipy.register_magics(CaptureMagic)

# Manim Example

In [ ]:
from manim import *
param= "-v WARNING  --progress_bar None  -r  500,200  --disable_caching Example"
config.media_embed = False

In [ ]:
%%capture_video -p "fromManim.mp4"
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        self.add(dot)
        self.wait()
        dot.scale(2)
        self.wait()
        dot.scale(2)
        self.wait(2)

# Web Example

In [ ]:
%%capture_video -p "fromWeb.mp4"
from IPython.display import Video
Video('https://github.com/kolibril13/jupyter_video_presentation/blob/master/001.mp4?raw=true' ,width=300)

In [ ]:
from pathlib import Path
Path("fromLocal.mp4").unlink()
Path("fromManim.mp4").unlink()
try:
    Path("fromWeb.mp4").unlink()
except:
    pass